In [5]:
import pandas
import os
import numpy as np

data = pandas.read_csv('features_SPY.csv', delimiter=',')
data['Date'] = pandas.to_datetime(data['Date'])

X = data.iloc[:, 1:23]
#print(X)

y = data['Date']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                      test_size=0.3, 
                                      random_state=241)

#y = np.where(data['Return_2'] >= 0, 1.0, 0.0)
#print(y[:20])
#y = np.roll(y, -2) # yes it's future returns, but last values in array - are random trash!
#print(y.shape)

In [10]:
days = [0, 1, 2, 3, 5, 10, 20, 40, 60, 120, 250]

def getTarget(period):
    return_key = "Return_" + str(period)
    Y = data[return_key]
    Y = np.where(Y >= 0, 1.0, 0.0)
    return np.roll(Y, -period)

#preparing features
def getFeaturesDataFrame(X, days, period):
    XfeaturesArray = __getFeatures(X, days, 2)
    XfeaturesMap = {"Feature" + str(i): XfeaturesArray[:,i] for i in range(0, XfeaturesArray[0].size)}
    return pandas.DataFrame(XfeaturesMap)

def __getFeatures(X, days, period):
    index_day = days.index(period)
    featureCount = (1 + days[index_day - 1]) * 2 + 2;
    
    
    #print("ii",index_day)
    return_key = "Return_" + str(period)
    volume_key = "AverageVolume_" + str(period)
    #print(return_key, volume_key)

    array = np.empty([0, featureCount])
    for index, row in X.iterrows():
        #if (index > 2): break;
        features = np.empty(0)
        #print(row)
        column = X[return_key]
        #print(column[index])
        #print("i",index)
        #print("a", features)
        features = np.append(features, column[index])
        #print("aa", features)
        for lag in range(1, days[index_day - 1] + 1):
            #print("lag = %d" % lag)
            #numpy.append(features, value
            if (index - lag >= 0):
                features = np.append(features, column[index - lag])
            else:
                features = np.append(features, float("NaN"))
        
        column = X[volume_key]
        features = np.append(features, column[index])
        for lag in range(1, days[index_day - 1] + 1):
            #print("lag = %d" % lag)
            #numpy.append(features, value
            if (index - lag >= 0):
                features = np.append(features, column[index - lag])
            else:
                features = np.append(features, float("NaN"))

        column = X['DayOfWeek']
        features = np.append(features, column[index])
        column = X['DayOfMonth']
        features = np.append(features, column[index])
        
        #print(features)
        #https://stackoverflow.com/a/30305148
        #print(features[None, :].shape)
        #print(array.shape)
        array = np.vstack([array, features[None, :]])
        #print("array",array)
    #print('\n')
    
    return array

#print(getFeatures(X, days, 2).shape)

In [22]:
def getTrainDataset(period):
    XFeatures = getFeaturesDataFrame(X, days, period)
    TrainFeatures = XFeatures.iloc[X_Train.index, :]
    TrainFeatures.reset_index(drop=True, inplace=True)
    return TrainFeatures

def getTrainTarget(period):
    Target = getTarget(period)
    Target = Target.iloc[y_train.index, :]
    Target.reset_index(drop=True, inplace=True)
    return Target

def getTestDataset(period):
    XFeatures = getFeaturesDataFrame(X, days, period)
    TestFeatures = XFeatures.iloc[X_test.index, :]
    TestFeatures.reset_index(drop=True, inplace=True)
    return TestFeatures

def getTestTarget(period):
    Target = getTarget(period)
    Target = Target.iloc[y_test.index, :]
    Target.reset_index(drop=True, inplace=True)
    return Target
#print(X_train.shape)
#print(X_test.shape)
#print(X_train)

#print(getTrainDataset(2))
#print(getTrainDataset(2).shape)
#print(getTarget(1))